In [1]:
import os.path
import time

import tensorflow as tf
import numpy as np
import random 

import layer_def as ld
import BasicConvLSTMCell

In [2]:
img_dim = (32,32) 
image_num = 3200
batch_size_ = 16
seq_len_ = 4
seq_start_ = 2

In [ ]:
flags = tf.app.flags
FLAGS = flags.FLAGS

tf.app.flags.DEFINE_string('f', '', 'kernel')


tf.app.flags.DEFINE_string('train_dir', './checkpoints/train_store_conv_lstm',
                            """dir to store trained net""")
tf.app.flags.DEFINE_integer('seq_length', 4,
                            """size of hidden layer""")
tf.app.flags.DEFINE_integer('seq_start', 2,
                            """ start of seq generation""")
tf.app.flags.DEFINE_integer('max_step', 200000,
                            """max num of steps""")
tf.app.flags.DEFINE_float('keep_prob', .8,
                            """for dropout""")
tf.app.flags.DEFINE_float('lr', .1,
                            """for dropout""")
tf.app.flags.DEFINE_integer('batch_size', 16,
                            """batch size for training""")
tf.app.flags.DEFINE_float('weight_init', .1,
                            """weight init for fully connected layers""")
FLAGS = flags.FLAGS

In [ ]:
def reshape_seq_len(data, seq_len = seq_len_):
    reshaped_x = []
    for x in data:
        temp = x.reshape([-1,seq_len,x.shape[1],x.shape[2]])
        reshaped_x.append(temp)
    return reshaped_x


def gen_batch(data, batch_size = batch_size_):
    X = data[0]
    Y = data[1]
    for i in range(0 , len(X), batch_size):
        yield X[i: i + batch_size], Y[i: i + batch_size]
        
        
def my_shuffle(data):
    
    suffled_x = []
    start_state = random.getstate()
    for x in data:
        random.shuffle(x)
        random.setstate(start_state)
        suffled_x.append(x)        
    return suffled_x    

In [ ]:
raw_X = np.asarray([np.ones(img_dim) for n in range(image_num)])
raw_Y = np.cumsum(raw_X,axis=0)

data = (raw_X, raw_X)
reshaped_data = reshape_seq_len(data, seq_len_)
shuffled_data = my_shuffle(reshaped_data)
data_generator = gen_batch(shuffled_data, batch_size_)

In [ ]:
def network(inputs, hidden, lstm=True):
    
    y_0 = inputs
    
    if lstm:
    # conv lstm cell 
        with tf.variable_scope('conv_lstm', initializer = tf.random_uniform_initializer(-.01, 0.1), reuse=tf.AUTO_REUSE):
                cell = BasicConvLSTMCell.BasicConvLSTMCell([32,32], [3,3], 1)
                if hidden is None:
                    hidden = cell.zero_state(FLAGS.batch_size, tf.float32) 
                y_1, hidden = cell(y_0, hidden)
    else:
        y_1 = ld.conv_layer(y_0, 3, 1, 8, "encode_3")

    return y_1, hidden

In [ ]:
def train():
  """Train ring_net for a number of steps."""
  with tf.Graph().as_default():
    # make inputs
    x = tf.placeholder(tf.float32, [None, seq_len_, 32, 32, 1])

    # possible dropout inside
    keep_prob = tf.placeholder("float")
    x_dropout = tf.nn.dropout(x, keep_prob)

    # create network
    x_unwrap = []

    # conv network
    hidden = None
    for i in range(seq_len_-1):
      if i < seq_start_:
        x_1, hidden = network_template(x_dropout[:,i,:,:,:], hidden)
      else:
        x_1, hidden = network_template(x_1, hidden)
      x_unwrap.append(x_1)

    # pack them all together 
    x_unwrap = tf.stack(x_unwrap)
    x_unwrap = tf.transpose(x_unwrap, [1,0,2,3,4])

    # this part will be used for generating video
    x_unwrap_g = []
    hidden_g = None
    for i in range(50):
      if i < seq_start_:
        x_1_g, hidden_g = network_template(x_dropout[:,i,:,:,:], hidden_g)
      else:
        x_1_g, hidden_g = network_template(x_1_g, hidden_g)
      x_unwrap_g.append(x_1_g)

    # pack them generated ones
    x_unwrap_g = tf.stack(x_unwrap_g)
    x_unwrap_g = tf.transpose(x_unwrap_g, [1,0,2,3,4])

    # calc total loss (compare x_t to x_t+1)
    loss = tf.nn.l2_loss(x[:,seq_start_+1:,:,:,:] - x_unwrap[:,seq_start_:,:,:,:])
    tf.summary.scalar('loss', loss)

    # training
    train_op = tf.train.AdamOptimizer(FLAGS.lr).minimize(loss)
    
    # List of all Variables
    variables = tf.global_variables()

    # Build a saver
    saver = tf.train.Saver(tf.global_variables())   

    # Summary op
    summary_op = tf.summary.merge_all()
    
    init = tf.global_variables_initializer()

    # Start running operations on the Graph.
    sess = tf.Session()

    
    print("init network from scratch")
    sess.run(init)

    # Summary op
    graph_def = sess.graph.as_graph_def(add_shapes=True)
    summary_writer = tf.summary.FileWriter(FLAGS.train_dir, graph_def=graph_def)

    for step in range(FLAGS.max_step):
      dat = data_generator.__next__()
      dat = np.expand_dims(dat[1], axis= 5)
      t = time.time()
      _, loss_r = sess.run([train_op, loss],feed_dict={x:dat, keep_prob:FLAGS.keep_prob})
      elapsed = time.time() - t

      if step%1 == 0 and step != 0:
        summary_str = sess.run(summary_op, feed_dict={x:dat, keep_prob:FLAGS.keep_prob})
        summary_writer.add_summary(summary_str, step) 
        print("time per batch is " + str(elapsed))
        print(step)
        print(loss_r)    
 

In [ ]:
network_template = tf.make_template('network', network)
train()